In [ ]:
!pip install -qq openai langchain pypdf tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.3 MB/s eta 0:00:00


5-1 문서 내용 로드

In [ ]:
from langchain.document_loaders import PyPDFLoader
def pdf_loader(pdf_path):
    loader = PyPDFLoader(pdf_path)
    pdf_doc = loader.load()
    return pdf_doc

In [ ]:
pdf_path = "hey_pdf.pdf"
pages_content = pdf_loader(pdf_path)
print(pages_content[0].page_content[:200])

안녕하세요
!
헤이마트는
신선한
과일과
다양한
상품을
판매하는
동네
마트입니다
.
고품질의
제품과
풍부한
다양성을
자랑하며
,
지역
고객들에게
최상의
서비스를
제공하기
위해
노력하고
있습니다
.
우리
마트에서는
신선한
과일뿐만
아니라
식료품
,
생활용품
,
주방용품
등
다양한
상품을
찾아보실
수
있습니다
.
고객들이
편리하게
쇼핑을
즐길
수
있도록
다양한



5-2 문서 내용 분할

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# 스플리터 설정
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=3000,
    chunk_overlap=0,
)
# pages_content 내용 분할
split_docs = text_splitter.split_documents(pages_content)
# 분할된 문서의 수 확인
print("분할된 문서의 개수는: ", len(split_docs))

분할된 문서의 개수는:  2


5-3 분할 내용 각각 요약 실행

In [ ]:
## 각각의 문서에 대한 요약 실행
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from openai import OpenAI
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# map template 설정, {pages_content} 분할된 내용이 입력
map_template = """다음은 문서 중 일부 내용입니다
{pages_content}
이 문서의 주요 내용을 요약해 주세요.
"""
# map 기본 프롬프트
map_prompt = PromptTemplate.from_template(map_template)
# 문서내용이 길 수 있기 때문에 model을 gpt-3.5-turbo-16k 설정
llm = ChatOpenAI(temperature=0,
                 model_name='gpt-3.5-turbo-16k')
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# reduce 단계에서 처리할 프롬프트 정의
reduce_template = """다음은 문서 요약의 집합입니다:
{summaries}
이 내용을 바탕으로 통합된 문서 요약을 작성해 주세요.
"""
# Reduce 프롬프트
reduce_prompt = PromptTemplate.from_template(reduce_template)

# Reduce에서 수행할 LLMChain 정의
reduce_chain = LLMChain(llm = llm, prompt = reduce_prompt)

5-4 각 문서의 요약본 통합 실행

In [ ]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import ReduceDocumentsChain

# 문서 목록 통합 체인 설정
combine_doc_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,
    document_variable_name="summaries" # reduce 프롬프트에 대입되는 변수
)

# 분할된 문서를 순차적으로 reduce 처리
reduce_doc_chain = ReduceDocumentsChain(
    combine_documents_chain=combine_doc_chain,
    collapse_documents_chain=combine_doc_chain,
    token_max=4000,
)

5-5 최종 결과 통합 체인 생성

In [ ]:
from langchain.chains import MapReduceDocumentsChain
# 최종 체인 연결
final_chain = MapReduceDocumentsChain(
    llm_chain=map_chain, # 각 문서 맵핑
    reduce_documents_chain=reduce_doc_chain,
    document_variable_name="pages_content",
    return_intermediate_steps=False,
)
# 최종 결과 실행
result_summary = final_chain.run(split_docs)
# 요약 결과 출력
print(result_summary)

헤이마트는 고품질의 제품과 다양성을 자랑하며 지역 고객들에게 최상의 서비스를 제공하기 위해 노력하고 있습니다. 마트에서는 과일뿐만 아니라 식료품, 생활용품, 주방용품 등 다양한 상품을 찾아볼 수 있으며, 편리한 쇼핑을 즐길 수 있도록 다양한 브랜드와 제품을 선보이고 있습니다. 헤이마트는 고객들의 다양한 쇼핑 요구를 충족시키기 위해 노력하고 있으며, 항상 최상의 제품과 서비스를 제공하여 고객들에게 만족스러운 경험을 선사하는 것을 목표로 하고 있습니다. 또한, 고객들의 의견을 듣고 더 나은 서비스를 제공하기 위해 노력하고 있으며, 지역 사회와 함께 성장하며, 지역의 발전에 기여하기 위해 노력할 것을 약속합니다. 헤이마트에 항상 관심과 사랑을 주셔서 감사합니다.
